# Análisis y limpieza de los datos

Importamos el train.csv

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
train_csv=pd.read_csv("../data/test.csv")

Primero vamos a analizar las columnas no numéricas a ver cuales podemos eliminar

In [3]:
non_numeric = train_csv.select_dtypes(include=object)
non_numeric.shape

(1389, 33)

Las primeras 7 columnas no nos dan información relevante por lo que las podemos quitar


In [4]:
columns_to_drop1=list(non_numeric.columns[0:8])

In [5]:
#Eliminamos columnas de texto no relevante (7 columnas)

train_mod=train_csv.drop(columns=columns_to_drop1,axis=1)

In [6]:
columns_to_drop2=list(train_mod.columns[0:3])

otras_columnas=["host_about","host_thumbnail_url", "host_picture_url", "calendar_last_scraped","license","neighbourhood_group_cleansed","bathrooms","calendar_updated"]

for i in otras_columnas:
    columns_to_drop2.append(i)


In [7]:
#Eliminamos columnas de texto no relevante  (columns_to_drop2)

train_mod=train_mod.drop(columns=columns_to_drop2,axis=1)

Como tenemos todos los valores para neighbourhood_cleansed, podemos elimnar la columna neighbourhood que tiene valores nulos

In [8]:
train_mod=train_mod.drop("neighbourhood",axis=1)

Nos encargamos de los nulos

In [9]:
# rellenamos con unknown

train_mod.host_response_time.fillna("unknown",inplace=True)
train_mod.host_neighbourhood.fillna("unknown",inplace=True)
train_mod.host_location.fillna("unknown",inplace=True)
train_mod.bathrooms_text.fillna("unknown",inplace=True)

In [10]:
#Para host_response_rate y host_acceptance_rate queremos 
#rellenar los NAN con la media, pero primero tenemos que quitar los signos "%" y pasarlo a float

train_mod.host_response_rate=train_mod.host_response_rate.str.replace("%","")
train_mod.host_acceptance_rate=train_mod.host_acceptance_rate.str.replace("%","")

train_mod.host_response_rate.fillna("0",inplace=True)
train_mod.host_acceptance_rate.fillna("0",inplace=True)

train_mod.host_response_rate.astype(dtype="int8")
train_mod.host_acceptance_rate.astype(dtype="int8")


train_mod.host_response_rate.fillna(train_mod.host_response_rate.mean(),inplace=True)
train_mod.host_acceptance_rate.fillna(train_mod.host_response_rate.mean(), inplace=True)



In [11]:
train_mod.head(2)

,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2013-10-22,"Amsterdam, North Holland, Netherlands",within an hour,100,50,f,unknown,1,1,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Oud-Oost,52.35713,4.92470,Private room in rental unit,Private room,1,1 shared bath,1.0,1.0,"[""Hangers"", ""Drying rack for clothing"", ""Freez...",2,1125,2.0,2.0,1125.0,1125.0,2.0,1125.0,t,29,59,89,89,2,2,2,2021-11-16,2021-11-07,5.0,5.0,5.0,5.0,5.0,5.0,5.0,f,1,0,1,0,2.0
1,2019-02-18,Netherlands,within an hour,100,100,f,unknown,0,0,"['email', 'phone']",t,t,Oud-Noord,52.38810,4.91438,Shared room in hostel,Shared room,2,1 shared bath,1.0,1.0,"[""Building staff"", ""Wifi"", ""Long term stays al...",1,365,1.0,3.0,999.0,999.0,1.0,999.0,t,0,0,5,280,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,12,0,10,2,NaN


### Columnas Reviews

Vamos a probar poniendo valor 0 a aquellas que no tienen review. Aunque esto no es cierto ya que parece que le estamos poniendo una puntuación de 0, la gente se guia mucho por estas review por lo que no tener afectará negativamente al preci (en teoria). Más adelante probaremos otras metodologías para ver con que asumciones es más preciso nuestro modelo. 

In [12]:
train_mod.first_review.fillna("0",inplace=True)               
train_mod.last_review.fillna("0",inplace=True)                         
train_mod.review_scores_rating.fillna("0",inplace=True)                 
train_mod.review_scores_accuracy.fillna("0",inplace=True)              
train_mod.review_scores_cleanliness.fillna("0",inplace=True)            
train_mod.review_scores_checkin.fillna("0",inplace=True)               
train_mod.review_scores_communication.fillna("0",inplace=True)          
train_mod.review_scores_location.fillna("0",inplace=True)               
train_mod.review_scores_value.fillna("0",inplace=True)                  
train_mod.reviews_per_month.fillna("0",inplace=True)      

In [13]:
nan_cols=train_mod.isna().sum()
nan_cols[nan_cols>0]

bedrooms    72
beds        77
dtype: int64

### Columnas Bedrooms y beds

Para estas dos columnas, vamos a probar primero poniendo la media a los valores nulos.

In [14]:
train_mod.bedrooms.fillna(train_mod.bedrooms.mean(),inplace=True)
train_mod.beds.fillna(train_mod.beds.mean(),inplace=True)

Para los nulos en las columnas minumum/maximum nights como solo hay uno, quitamos esa fila ya que todos están en la misma

In [15]:
train_mod.dropna(axis=0,inplace=True)

In [16]:
train_mod=train_mod.drop(columns=["host_since","first_review"],axis=1)

In [17]:
train_mod.head(2)


,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,"Amsterdam, North Holland, Netherlands",within an hour,100,50,f,unknown,1,1,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Oud-Oost,52.35713,4.92470,Private room in rental unit,Private room,1,1 shared bath,1.0,1.0,"[""Hangers"", ""Drying rack for clothing"", ""Freez...",2,1125,2.0,2.0,1125.0,1125.0,2.0,1125.0,t,29,59,89,89,2,2,2,2021-11-07,5.0,5.0,5.0,5.0,5.0,5.0,5.0,f,1,0,1,0,2.0
1,Netherlands,within an hour,100,100,f,unknown,0,0,"['email', 'phone']",t,t,Oud-Noord,52.38810,4.91438,Shared room in hostel,Shared room,2,1 shared bath,1.0,1.0,"[""Building staff"", ""Wifi"", ""Long term stays al...",1,365,1.0,3.0,999.0,999.0,1.0,999.0,t,0,0,5,280,0,0,0,0,0,0,0,0,0,0,0,t,12,0,10,2,0


# Columnas anidadas (host_verifications y amenities) 

#Como un primer approach voy a poner estas columnas como escala, contanto el numero de ammenities/verifications que tiene. En otros modelos probare separando las categorias y poniendo columnas binarias



## Exportamos datos a utilizar

En el primer caso solo vamos a usar las variables numéricas de la tabla ya modificada.


In [18]:
#cogemos solo valores numericos:

test_mod_numeric= train_mod._get_numeric_data()

In [19]:
test_mod_numeric.to_csv("../data/outcomes_models/test_mod_numeric.csv")

In [20]:
test_limp1=train_mod

In [21]:
test_limp1.to_csv("../data/outcomes_models/test_limp1.csv")